
     Fast Start Example #4 - RAG with Text Query<br>
    This example shows a basic RAG recipe using text query combined with LLM prompt.<br>
    We will show two different ways to achieve this basic recipe:<br>
    -- Example 4A - this will integrate Library + Prompt - and is the most scalable general solution<br>
    -- Example 4B - this will illustrate another capability of the Prompt class to add sources "inline"<br>
     without necessarily a library in-place.  It is another useful tool when you want to be able to quickly<br>
     pick up a document and start asking questions to it.<br>
     Note: both of the examples are designed to achieve the same output.<br>


In [1]:
import os
import re
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.setup import Setup
from llmware.configs import LLMWareConfig
from llmware.retrieval import Query
from llmware.library import Library

In [2]:
# Replace 'YOUR_API_KEY' with your actual Hugging Face API key
os.environ["HF_HOME"] = "/root/.cache/huggingface"
os.environ["HF_HOME"] = "/root/.huggingface"
os.environ["HF_HOME"] = "/root/.huggingface"
os.environ["USER_HOME"] = "/root"


api_key = os.getenv("YOUR_API_KEY")

In [3]:
def example_4a_contract_analysis_from_library (model_name, verbose=False):
    """ Example #4a:  Main general case to run a RAG workflow from a Library """

    # Load the llmware sample files
    print (f"\n > Loading the llmware sample files...")
    sample_files_path = Setup().load_sample_files()
    contracts_path = os.path.join(sample_files_path,"Agreements")
    contracts_lib = Library().create_new_library("example4_library")
    contracts_lib.add_files(contracts_path)

    # questions that we want to ask each contract
    question_list = [{"topic": "executive employment agreement", "llm_query": "What are the names of the two parties?"},
                     {"topic": "base salary", "llm_query": "What is the executive's base salary?"},
                     {"topic": "governing law", "llm_query": "What is the governing law?"}]
    print (f"\n > Loading model {model_name}...")
    q = Query(contracts_lib)

    # get a list of all of the unique documents in the library

    # doc id list
    doc_list = q.list_doc_id()
    print("update: document id list - ", doc_list)

    # filename list
    fn_list = q.list_doc_fn()
    print("update: filename list - ", fn_list)
    prompter = Prompt().load_model(model_name)
    for i, doc_id in enumerate(doc_list):
        print("\nAnalyzing contract: ", str(i+1), doc_id, fn_list[i])
        print("LLM Responses:")
        for question in question_list:
            query_topic = question["topic"]
            llm_question = question["llm_query"]
            doc_filter = {"doc_ID": [doc_id]}
            query_results = q.text_query_with_document_filter(query_topic,doc_filter,result_count=5,exact_mode=True)
            if verbose:
                # this will display the query results from the query above
                for j, qr in enumerate(query_results):
                    print("update: querying document - ", query_topic, j, doc_filter, qr)
            source = prompter.add_source_query_results(query_results)

            #   *** this is the call to the llm with the source packaged in the context automatically ***
            responses = prompter.prompt_with_source(llm_question, prompt_name="default_with_context", temperature=0.3)

            #   unpacking the results from the LLM
            for r, response in enumerate(responses):
                print("update: llm response -  ", llm_question, re.sub("[\n]"," ", response["llm_response"]).strip())

            # We're done with this contract, clear the source from the prompt
            prompter.clear_source_materials()

    #   Save jsonl report to jsonl to /prompt_history folder
    print("\nPrompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))
    prompter.save_state()

    #   Save csv report that includes the model, response, prompt, and evidence for human-in-the-loop review
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
    print("\nCSV output saved at:  ", csv_output)
    return 0

In [5]:
def example_4b_contract_analysis_direct_from_prompt(model_name, verbose=False):
    """ Example #4b: Alternative implementation using prompt in-line capabilities without using a library """

    # Load the llmware sample files
    print(f"\n > Loading the llmware sample files...")
    sample_files_path = Setup().load_sample_files()
    contracts_path = os.path.join(sample_files_path, "Agreements")

    # questions that we want to ask each contract
    question_list = [{"topic": "executive employment agreement", "llm_query": "What are the names of the two parties?"},
                     {"topic": "base salary", "llm_query": "What is the executive's base salary?"},
                     {"topic": "governing law", "llm_query": "What is the governing law?"}]
    print(f"\n > Loading model {model_name}...")
    prompter = Prompt().load_model(model_name)
    for i, contract in enumerate(os.listdir(contracts_path)):

        # exclude potential mac os created file artifact in the samples folder path
        if contract != ".DS_Store":
            print("\nAnalyzing contract: ", str(i + 1), contract)
            print("LLM Responses:")
            for question in question_list:
                query_topic = question["topic"]
                llm_question = question["llm_query"]
                #   introducing "add_source_document"
                #   this will perform 'inline' parsing, text chunking and query filter on a document
                #   input is a file folder path, file name, and an optional query filter
                #   the source is automatically packaged into the prompt object
                source = prompter.add_source_document(contracts_path,contract,query=query_topic)
                if verbose:
                    print("update: document created source - ", source)
                #   calling the LLM with 'source' information from the contract automatically packaged into the prompt
                responses = prompter.prompt_with_source(llm_question, prompt_name="default_with_context",
                                                        temperature=0.3)
                #   unpacking the LLM responses
                for r, response in enumerate(responses):
                    print("update: llm response: ", llm_question, re.sub("[\n]", " ",
                                                                         response["llm_response"]).strip())
                # We're done with this contract, clear the source from the prompt
                prompter.clear_source_materials()

    # Save jsonl report to jsonl to /prompt_history folder
    print("\nupdate: Prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(), prompter.prompt_id))
    prompter.save_state()

    # Save csv report that includes the model, response, prompt, and evidence for human-in-the-loop review
    csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
    print("\nupdate: CSV output saved at - ", csv_output)
    return 0

if __name__ == "__main__":

    #   you can pick any model from the ModelCatalog
    #   we list a few representative good choices below

    LLMWareConfig().set_active_db("sqlite")

    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"

    # use local cpu model
    model_name = example_models[0]

    #   two good recipes to address the use case

    #   first let's look at the main way of retrieving and analyzing from a library
    example_4a_contract_analysis_from_library(model_name)

    #   second - uncomment this line, and lets run the "in-line" prompt way
    # example_4b_contract_analysis_direct_from_prompt(model_name)




 > Loading the llmware sample files...

 > Loading model llmware/bling-1b-0.1...
update: document id list -  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
update: filename list -  ['Amphitrite EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Aphrodite EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Apollo EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Artemis Poseidon EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Athena EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Bia EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Demeter EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Eileithyia EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Gaia EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Leto EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Metis EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Nike EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Nyx EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Persephone EXECUTIVE EMPLOYMENT AGREEMENT.pdf', 'Rhea EXECUTIVE EMPLOYMENT AGREEMENT.pdf']


C:\Users\hsyyu\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

C:\Users\hsyyu\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hsyyu\.cache\huggingface\hub\models--llmware--bling-1b-0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\hsyyu\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132:

pytorch_model.bin:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]


Analyzing contract:  1 1 Amphitrite EXECUTIVE EMPLOYMENT AGREEMENT.pdf
LLM Responses:
update: llm response -   What are the names of the two parties? Amphitrite Ares and TestCo Software, Inc.
update: llm response -   What is the executive's base salary? $5,000,000.
update: llm response -   What is the governing law? State of Massachusetts

Analyzing contract:  2 2 Aphrodite EXECUTIVE EMPLOYMENT AGREEMENT.pdf
LLM Responses:
update: llm response -   What are the names of the two parties? Aphrodite Apollo and TestCo Software, Inc.
update: llm response -   What is the executive's base salary? $600,000.
update: llm response -   What is the governing law? State of Massachusetts

Analyzing contract:  3 3 Apollo EXECUTIVE EMPLOYMENT AGREEMENT.pdf
LLM Responses:
update: llm response -   What are the names of the two parties? Aphrodite Apollo and TestCo Software, Inc.
update: llm response -   What is the executive's base salary? $600,000.
update: llm response -   What is the governing law? Stat

In [ ]:

"""     Fast Start Example #5 - RAG with Semantic Query

    This example illustrates the most common RAG retrieval pattern, which is using a semantic query, e.g.,
    a natural language query, as the basis for retrieving relevant text chunks, and then using as
    the context material in a prompt to ask the same question to a LLM.

    In this example, we will show the following:

    1.  Create library and install embeddings (feel free to skip / substitute a library created in an earlier step).
    2.  Ask a general semantic query to the entire library collection.
    3.  Select the most relevant results by document.
    4.  Loop through all of the documents - packaging the context and asking our questions to the LLM.

    NOTE: to use chromadb, you may need to install the python sdk:  pip3 install chromadb.

"""


import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.prompts import Prompt
from llmware.configs import LLMWareConfig
from importlib import util
if not util.find_spec("chromadb"):
    print("\nto run this example with chromadb, you need to install the chromadb python sdk:  pip3 install chromadb")


def semantic_rag (library_name, embedding_model_name, llm_model_name):

    """ Illustrates the use of semantic embedding vectors in a RAG workflow
        --self-contained example - will be duplicative with some of the steps taken in other examples """

    # Step 1 - Create library which is the main 'organizing construct' in llmware
    print ("\nupdate: Step 1 - Creating library: {}".format(library_name))

    library = Library().create_new_library(library_name)

    # Step 2 - Pull down the sample files from S3 through the .load_sample_files() command
    #   --note: if you need to refresh the sample files, set 'over_write=True'
    print ("update: Step 2 - Downloading Sample Files")

    sample_files_path = Setup().load_sample_files(over_write=False)
    contracts_path = os.path.join(sample_files_path, "Agreements")

    # Step 3 - point ".add_files" method to the folder of documents that was just created
    #   this method parses all of the documents, text chunks, and captures in MongoDB
    print("update: Step 3 - Parsing and Text Indexing Files")

    library.add_files(input_folder_path=contracts_path, chunk_size=400, max_chunk_size=600,
                      smart_chunking=1)

    # Step 4 - Install the embeddings
    print("\nupdate: Step 4 - Generating Embeddings in {} db - with Model- {}".format(vector_db, embedding_model))

    library.install_new_embedding(embedding_model_name=embedding_model_name, vector_db=vector_db)

    # RAG steps start here ...

    print("\nupdate: Loading model for LLM inference - ", llm_model_name)

    prompter = Prompt().load_model(llm_model_name)

    query = "what is the executive's base annual salary"

    #   key step: run semantic query against the library and get all of the top results
    results = Query(library).semantic_query(query, result_count=50, embedding_distance_threshold=1.0)

    #   if you want to look at 'results', uncomment the two lines below
    #   for i, res in enumerate(results):
    #       print("update: ", i, res["file_source"], res["distance"], res["text"])

    for i, contract in enumerate(os.listdir(contracts_path)):

        qr = []

        if contract != ".DS_Store":

            print("\nContract Name: ", i, contract)

            #   we will look through the list of semantic query results, and pull the top results for each file
            for j, entries in enumerate(results):

                library_fn = entries["file_source"]
                if os.sep in library_fn:
                    # handles difference in windows file formats vs. mac / linux
                    library_fn = library_fn.split(os.sep)[-1]

                if library_fn == contract:
                    print("Top Retrieval: ", j, entries["distance"], entries["text"])
                    qr.append(entries)

            #   we will add the query results to the prompt
            source = prompter.add_source_query_results(query_results=qr)

            #   run the prompt
            response = prompter.prompt_with_source(query, prompt_name="default_with_context", temperature=0.3)

            #   note: prompt_with_resource returns a list of dictionary responses
            #   -- depending upon the size of the source context, it may call the llm several times
            #   -- each dict entry represents 1 call to the LLM

            for resp in response:
                if "llm_response" in resp:
                    print("\nupdate: llm answer - ", resp["llm_response"])

            # start fresh for next document
            prompter.clear_source_materials()

    return 0


if __name__ == "__main__":

    LLMWareConfig().set_active_db("sqlite")

    #   for this example, we will use an embedding model that has been 'fine-tuned' for contracts
    embedding_model = "industry-bert-contracts"

    #   note: as of llmware==0.2.12, we have shifted from faiss to chromadb for the Fast Start examples
    #   --if you are using a Python version before 3.12, please feel free to substitute for "faiss"
    #   --for versions of Python >= 3.12, for the Fast Start examples (e.g., no install required), we
    #   recommend using chromadb or lancedb

    #   please double-check: `pip3 install chromadb` or pull the latest llmware version to get automatically
    #   -- if you have installed any other vector db, just change the name, e.g, "milvus" or "pg_vector"

    vector_db = "chromadb"

    # pick any name for the library
    lib_name = "example_5_library"

    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]

    # use local cpu model
    llm_model_name = example_models[0]

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   llm_model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"

    semantic_rag(lib_name, embedding_model, llm_model_name)



